<a href="https://colab.research.google.com/github/Wuwei0-1/-github-workflows-lucid_dream.-yml/blob/main/Website.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
# STEP 1: INSTALL CRYPTOGRAPHY LIBRARY
# (This runs silently to get the security tools ready)
!pip install cryptography -q

import hashlib
import datetime
from cryptography.fernet import Fernet
import getpass

# --- THE ARCHITECTURE ---

class SecureVault:
    def __init__(self):
        # This acts as our "Database"
        self.users = {}
        self.inventions = {}
        self.access_ledger = {} # Who is allowed to see what
        self.audit_log = [] # The "Trustee" view

        # Generate a Master Key for the system (In real life, this is stored in a Hardware Security Module)
        self.key = Fernet.generate_key()
        self.cipher = Fernet(self.key)

    # --- USER MANAGEMENT ---
    def create_user(self, username, role):
        valid_roles = ["Inventor", "Investor", "Engineer", "Trustee"]
        if role not in valid_roles:
            return "Error: Invalid Role"

        self.users[username] = {"role": role, "reputation": 100}
        self.log_event(f"User Created: {username} ({role})")
        return f"Welcome, {username}."

    # --- THE SECURE UPLOAD (ENCRYPTION) ---
    def upload_invention(self, owner_name, title, secret_details):
        if self.users[owner_name]["role"] != "Inventor":
            return "Error: Only Inventors can upload."

        # 1. Encrypt the secret data
        encrypted_data = self.cipher.encrypt(secret_details.encode())

        # 2. Create a Digital Fingerprint (Hash) for proof of time/content
        # This replaces the Blockchain. If one letter changes, this ID changes.
        unique_id = hashlib.sha256((title + owner_name + str(datetime.datetime.now())).encode()).hexdigest()[:10]

        self.inventions[unique_id] = {
            "owner": owner_name,
            "title": title,
            "data": encrypted_data, # WE STORE IT ENCRYPTED
            "timestamp": str(datetime.datetime.now())
        }

        # 3. Grant access to the owner automatically
        self.access_ledger[unique_id] = [owner_name]

        self.log_event(f"Invention Uploaded: {title} by {owner_name}. ID: {unique_id}")
        return f"Success! Invention ID: {unique_id}"

    # --- THE HANDSHAKE (PERMISSION) ---
    def grant_access(self, owner_name, requestor_name, invention_id):
        # Security Check: Only the owner can grant access
        if self.inventions[invention_id]["owner"] != owner_name:
            return "Error: You do not own this invention."

        self.access_ledger[invention_id].append(requestor_name)
        self.log_event(f"ACCESS GRANTED: {owner_name} -> {requestor_name} for Item {invention_id}")
        return f"Access granted to {requestor_name}."

    # --- VIEWING THE DATA (DECRYPTION) ---
    def view_invention(self, user_name, invention_id):
        # 1. Check if Invention exists
        if invention_id not in self.inventions:
            return "Error: Invention not found."

        # 2. CHECK PERMISSIONS (The Gatekeeper)
        allowed_users = self.access_ledger.get(invention_id, [])
        user_role = self.users[user_name]["role"]

        # Trustee Rule: Can see METADATA but NOT content
        if user_role == "Trustee":
            meta = self.inventions[invention_id]
            return f"[TRUSTEE VIEW] ID: {invention_id} | Owner: {meta['owner']} | Title: {meta['title']} | CONTENT: [LOCKED/ENCRYPTED]"

        # Regular User Rule
        if user_name not in allowed_users:
            self.log_event(f"SECURITY ALERT: {user_name} tried to access {invention_id} denied.")
            return "ACCESS DENIED. You do not have permission."

        # 3. Decrypt data if allowed
        encrypted_blob = self.inventions[invention_id]["data"]
        decrypted_secret = self.cipher.decrypt(encrypted_blob).decode()

        return f"--- OPENING SECURE FILE ---\nTitle: {self.inventions[invention_id]['title']}\nSecret Content: {decrypted_secret}\n---------------------------"

    # --- AUDIT LOG ---
    def log_event(self, message):
        timestamp = str(datetime.datetime.now())
        self.audit_log.append(f"[{timestamp}] {message}")

# ==========================================
# LET'S RUN THE SIMULATION
# ==========================================

system = SecureVault()


print("--- 1. CREATING PROFILES ---")
print(system.create_user("Alice", "Inventor"))   # Profile 1
print(system.create_user("Charlie", "Investor")) # Profile 2
print(system.create_user("BobCorp", "Engineer")) # Profile 3
print(system.create_user("Dave", "Trustee"))     # Profile 4

print("\n--- 2. INVENTOR UPLOADS IDEA ---")
# Alice uploads her secret. In memory, this is now encrypted bytes.
upload_msg = system.upload_invention("Alice", "Wuwei01", "The secret formula is H2O + Vibranium.")
print(upload_msg)
# Extract the ID from the message for the demo
inv_id = upload_msg.split(": ")[1]

print("\n--- 3. UNAUTHORIZED ACCESS ATTEMPT ---")
# BobCorp tries to look without asking.
print(system.view_invention("BobCorp", inv_id))

print("\n--- 4. THE HANDSHAKE ---")
# Alice grants Bob permission.
print(system.grant_access("Alice", "BobCorp", inv_id))

print("\n--- 5. AUTHORIZED ACCESS ---")
# BobCorp tries again.
print(system.view_invention("BobCorp", inv_id))

print("\n--- 6. TRUSTEE AUDIT ---")
# Dave (Trustee) checks the file. He should see it exists, but NOT the secret formula.
print(system.view_invention("Dave", inv_id))

print("\n--- 7. SYSTEM LOGS (What the AI watches) ---")
for log in system.audit_log:
    print(log)

--- 1. CREATING PROFILES ---
Welcome, Alice.
Welcome, Charlie.
Welcome, BobCorp.
Welcome, Dave.

--- 2. INVENTOR UPLOADS IDEA ---
Success! Invention ID: 799400e5a0

--- 3. UNAUTHORIZED ACCESS ATTEMPT ---
ACCESS DENIED. You do not have permission.

--- 4. THE HANDSHAKE ---
Access granted to BobCorp.

--- 5. AUTHORIZED ACCESS ---
--- OPENING SECURE FILE ---
Title: Wuwei01
Secret Content: The secret formula is H2O + Vibranium.
---------------------------

--- 6. TRUSTEE AUDIT ---
[TRUSTEE VIEW] ID: 799400e5a0 | Owner: Alice | Title: Wuwei01 | CONTENT: [LOCKED/ENCRYPTED]

--- 7. SYSTEM LOGS (What the AI watches) ---
[2025-12-13 09:27:55.055192] User Created: Alice (Inventor)
[2025-12-13 09:27:55.055269] User Created: Charlie (Investor)
[2025-12-13 09:27:55.055319] User Created: BobCorp (Engineer)
[2025-12-13 09:27:55.055364] User Created: Dave (Trustee)
[2025-12-13 09:27:55.055696] Invention Uploaded: Wuwei01 by Alice. ID: 799400e5a0
[2025-12-13 09:27:55.055936] SECURITY ALERT: BobCorp tri

In [10]:
# 1. Install the UI library (Runs silently)
!pip install gradio -q

import gradio as gr

# We reuse the System you built in the previous step
# (Make sure you ran the previous cell first!)
# We are just connecting "Buttons" to your "Functions"

def ui_create_user(name, role):
    return system.create_user(name, role)

def ui_upload(name, title, content):
    return system.upload_invention(name, title, content)

def ui_grant(owner, investor, id):
    return system.grant_access(owner, investor, id)

def ui_view(name, id):
    return system.view_invention(name, id)

def ui_audit():
    # Returns the last 10 logs for the Trustee
    return "\n".join(system.audit_log[-10:])

# --- BUILDING THE LAYOUT ---
with gr.Blocks(theme=gr.themes.Soft()) as app:
    gr.Markdown("# 🔒 Secure Invention Database")
    gr.Markdown("An encrypted repository for IP protection.")

    # TAB 1: REGISTRATION
    with gr.Tab("1. Create Profile"):
        gr.Markdown("### Sign Up")
        with gr.Row():
            reg_name = gr.Textbox(label="Username")
            reg_role = gr.Dropdown(["Inventor", "Investor", "Engineer", "Trustee"], label="Select Role")
        reg_btn = gr.Button("Create Account")
        reg_output = gr.Textbox(label="System Status")

        reg_btn.click(ui_create_user, inputs=[reg_name, reg_role], outputs=reg_output)

    # TAB 2: INVENTOR ZONE
    with gr.Tab("2. Inventor Zone"):
        gr.Markdown("### Upload New Invention (Encrypted)")
        with gr.Row():
            inv_name = gr.Textbox(label="Your Username (Must match login)")
            inv_title = gr.Textbox(label="Invention Title")
        inv_content = gr.Textbox(label="Secret Details / Prototype Link", lines=3)
        upload_btn = gr.Button("Encrypt & Upload")
        upload_output = gr.Textbox(label="Result (Copy the ID!)")

        upload_btn.click(ui_upload, inputs=[inv_name, inv_title, inv_content], outputs=upload_output)

        gr.Markdown("---")
        gr.Markdown("### Grant Access (The Handshake)")
        with gr.Row():
            grant_owner = gr.Textbox(label="Your Username")
            grant_investor = gr.Textbox(label="Investor Username")
            grant_id = gr.Textbox(label="Invention ID")
        grant_btn = gr.Button("Grant Permission")
        grant_output = gr.Textbox(label="Status")

        grant_btn.click(ui_grant, inputs=[grant_owner, grant_investor, grant_id], outputs=grant_output)

    # TAB 3: MARKETPLACE (VIEWER)
    with gr.Tab("3. View Invention"):
        gr.Markdown("### Secure Viewer")
        with gr.Row():
            view_name = gr.Textbox(label="Your Username")
            view_id = gr.Textbox(label="Invention ID")
        view_btn = gr.Button("Attempt Access")
        view_output = gr.Textbox(label="Decrypted Content", lines=5)

        view_btn.click(ui_view, inputs=[view_name, view_id], outputs=view_output)

    # TAB 4: TRUSTEE DASHBOARD
    with gr.Tab("4. Trustee Logs"):
        gr.Markdown("### System Audit Log (No Secrets Visible)")
        refresh_btn = gr.Button("Refresh Logs")
        log_output = gr.Textbox(label="Audit Trail", lines=10)

        refresh_btn.click(ui_audit, inputs=None, outputs=log_output)

# Launch the App
app.launch()

/tmp/ipython-input-2678412433.py:27: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(theme=gr.themes.Soft()) as app:


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://eff10ca8b430284666.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
